<a href="https://colab.research.google.com/github/Vasilis-Kyriakopoulos/Finetune-LLM-/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# from datasets import load_dataset
# import pandas as pd


# ds = load_dataset("maxscha/commitbench")
# main_path = "data/"

# cols = ["diff", "message"]


# ds["train"].select_columns(cols) \
#     .to_pandas() \
#     .to_csv(main_path+"commitbench_train.csv", index=False)


# ds["validation"].select_columns(cols) \
#     .to_pandas() \
#     .to_csv(main_path+"commitbench_validation.csv",index=False)


# ds["test"].select_columns(cols) \
#     .to_pandas() \
#     .to_csv(main_path+"commitbench_test.csv", index=False)


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/225M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/225M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
#from google.colab import drive
import pandas as pd


main_path = "data/"
train_df = pd.read_csv(main_path + "commitbench_train_10pct.csv")
val_df = pd.read_csv(main_path + "commitbench_validation_10pct.csv")
print(f"Length Train:{len(train_df)}")
print(f"Length Val:{len(val_df)}")

train_df = train_df[train_df["diff"].str.len() < 1000].reset_index(drop=True)
val_df = val_df[val_df["diff"].str.len() < 1000].reset_index(drop=True)
train_df = train_df[~train_df['message'].str.contains('^Fixes #', na=False)]

print(f"New Length Train: {len(train_df)}")
print(f"New Length Val: {len(val_df)}")

Length Train:116521
Length Val:24969
New Length Train: 92181
New Length Val: 19706


In [19]:
# import pandas as pd

# input_csv = main_path + "commitbench_validation.csv"
# output_csv = main_path + "commitbench_validation_10pct.csv"

# chunk_size = 100_000
# keep_frac = 0.1
# first = True

# for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
#     sampled = chunk.sample(frac=keep_frac, random_state=42)

#     sampled.to_csv(
#         output_csv,
#         mode="a",
#         index=False,
#         header=first
#     )
#     first = False


# input_csv = main_path + "commitbench_train.csv"
# output_csv = main_path + "commitbench_train_10pct.csv"

# chunk_size = 100_000
# keep_frac = 0.1
# first = True

# for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
#     sampled = chunk.sample(frac=keep_frac, random_state=42)

#     sampled.to_csv(
#         output_csv,
#         mode="a",
#         index=False,
#         header=first
#     )
#     first = False


In [4]:
import re

# 1. Λήψη του πρώτου δείγματος (πρώτη γραμμή, στήλη 'diff')
#text = train_df.iloc[0]['diff']
initial_text =' https://gith'
text = 'https://gith'

# 1. Αφαίρεση της γραμμής index (είτε έχει πραγματικό hash είτε το λεκτικό <HASH>)
    # Χρησιμοποιούμε το .* για να πιάσουμε τα πάντα μέχρι την αλλαγή γραμμής
text = re.sub(r'^index .*\n', '', text, flags=re.MULTILINE)
    
    # 2. Απλοποίηση του 'diff --git' σε 'FILE:'
    # Κρατάμε μόνο το όνομα του αρχείου. Είναι το πιο σημαντικό context!
text = re.sub(r'^diff --git a/(.*) b/(.*)\n', r'FILE: \1\n', text, flags=re.MULTILINE)
    
    # 3. Αφαίρεση των γραμμών --- a/ και +++ b/ 
    # Αφού έχουμε το FILE:, αυτές οι γραμμές είναι 100% περιττές.
text = re.sub(r'^--- a/.*\n', '', text, flags=re.MULTILINE)
text = re.sub(r'^\+\+\+ b/.*\n', '', text, flags=re.MULTILINE)

    # 4. Αφαίρεση Git metadata από το commit message (αν η συνάρτηση εφαρμόζεται και εκεί)
text = re.sub(r'^(Signed-off-by|Co-authored-by|Reported-by):.*$', '', text, flags=re.MULTILINE)

    # 5. Καθαρισμός κενών γραμμών που προέκυψαν από τις αφαιρέσεις
text = re.sub(r'\n\s*\n', '\n', text)

text = re.sub(r'https?://\S+', '', text, flags=re.MULTILINE)
print("--- ΑΡΧΙΚΟ DIFF ---")
print(initial_text)

print("\n--- ΚΑΘΑΡΙΣΜΕΝΟ DIFF ---")
print(text)

--- ΑΡΧΙΚΟ DIFF ---
 https://gith

--- ΚΑΘΑΡΙΣΜΕΝΟ DIFF ---



In [3]:
train_df.head()

,diff,message
0,diff --git a/lib/bibreformat.py b/lib/bibrefor...,BibFormat: HDREF processing bug fix\n\n* Fixes...
1,diff --git a/src/lib/Supra/Controller/Pages/Tw...,Task #<I>: Block titles\nfast-fix implementati...
2,diff --git a/test/unit/serializers/cbor.spec.j...,Add additional cbor test for useTag<I>ForMaps
3,diff --git a/setup.py b/setup.py\nindex <HASH>...,[build] Add tests' requires in setup.py
4,diff --git a/lib/macho/load_commands.rb b/lib/...,map lazy/upward dylib to proper load command\n...


In [4]:
val_df.head()

,diff,message
0,diff --git a/redisson/src/main/java/org/rediss...,Fixed - Redisson cluster cannot recover if Red...
1,diff --git a/translate/coroutines.py b/transla...,bug fix when transliteration doesnt exist prin...
2,diff --git a/setup.py b/setup.py\nindex <HASH>...,add feature_selection to setup.py
3,diff --git a/src/resources/views/admin/_index....,.btn outside of title h1
4,diff --git a/Db.php b/Db.php\nindex <HASH>..<H...,Accept empty condition in smartSelect


In [29]:
# import numpy as np
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
# tokenizer.pad_token = tokenizer.eos_token

# def format_input(entry):

#     input_text = f"{entry['diff']}\nCommit Message:\n{entry['message']}"

#     return input_text

# def token_len(t: str) -> int:
#     return len(tokenizer.encode(t, add_special_tokens=True))


# rng = np.random.default_rng(42)
# texts = train_df.apply(format_input, axis=1).tolist()
# sample_size = min(50_000, len(texts))
# sample_texts = rng.choice(texts, size=sample_size, replace=False)

# lengths = np.array([token_len(t) for t in sample_texts], dtype=np.int32)

# p = [50, 75, 90, 95, 97, 98, 99, 99.5]
# qs = np.percentile(lengths, p)

# for perc, q in zip(p, qs):
#     print(f"{perc:>5}%  -> {int(q)} tokens")

# # Example choice:
# max_len = int(np.percentile(lengths, 95))
# print("Suggested max_len:", max_len)


   50%  -> 293 tokens
   75%  -> 363 tokens
   90%  -> 423 tokens
   95%  -> 458 tokens
   97%  -> 479 tokens
   98%  -> 496 tokens
   99%  -> 524 tokens
 99.5%  -> 550 tokens
Suggested max_len: 458


In [9]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import re

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(tokenizer.encode("<|endoftext|>"))

def clean_text(text):
# 1. Αφαίρεση της γραμμής index (είτε έχει πραγματικό hash είτε το λεκτικό <HASH>)
    # Χρησιμοποιούμε το .* για να πιάσουμε τα πάντα μέχρι την αλλαγή γραμμής
    text = re.sub(r'^index .*\n', '', text, flags=re.MULTILINE)
    
    # 2. Απλοποίηση του 'diff --git' σε 'FILE:'
    # Κρατάμε μόνο το όνομα του αρχείου. Είναι το πιο σημαντικό context!
    text = re.sub(r'^diff --git a/(.*) b/(.*)\n', r'FILE: \1\n', text, flags=re.MULTILINE)
    
    # 3. Αφαίρεση των γραμμών --- a/ και +++ b/ 
    # Αφού έχουμε το FILE:, αυτές οι γραμμές είναι 100% περιττές.
    text = re.sub(r'^--- a/.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\+\+\+ b/.*\n', '', text, flags=re.MULTILINE)


    # 5. Καθαρισμός κενών γραμμών που προέκυψαν από τις αφαιρέσεις
    text = re.sub(r'\n\s*\n', '\n', text)

    # 1. Αφαίρεση metadata (Signed-off-by, Co-authored-by, κλπ)
    # Αυτά καταστρέφουν το training γιατί το μοντέλο μαθαίνει ονόματα αντί για κώδικα
    text = re.sub(r'^(Signed-off-by|Co-authored-by|Reported-by|Reviewed-by|Cc):.*$', '', text, flags=re.MULTILINE)
    
    # 2. Αφαίρεση links προς Issues ή Pull Requests (π.χ. https://github.com...)
    # Τα URL είναι τεράστια σε tokens και δεν προσφέρουν νόημα στο GPT-2
    text = re.sub(r'https?://\S+', '', text, flags=re.MULTILINE)
    
    # 3. Αφαίρεση των placeholders <HASH> ή <I> αν υπάρχουν μόνα τους
    text = text.replace('<HASH>', '').replace('<I>', '')
    
    # 4. Καθαρισμός πολλαπλών κενών και αλλαγών γραμμής
    #text = re.sub(r'\s+', ' ', text)

    return text.strip()


class CodeDiffMessageDataset(Dataset):
    def __init__(self, data, tokenizer,max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = 512


    def __getitem__(self, index):
        entry = self.data.iloc[index]
        
        # Clean the components
        clean_diff = clean_text(entry['diff'])
        clean_msg = clean_text(entry['message'])

        # Structure: <|endoftext|> DIFF: ... MESSAGE: ... <|endoftext|>
        prompt = f"<|endoftext|>DIFF:\n{clean_diff}\n\nCOMMIT MESSAGE:\n"
        full_text = f"{prompt}{clean_msg}<|endoftext|>"

        encoded_full = self.tokenizer.encode(full_text, max_length=self.max_length, truncation=True)
        encoded_prompt = self.tokenizer.encode(prompt, max_length=self.max_length, truncation=True)
        
        return  encoded_full


    def __len__(self):
        return int(len(self.data))

train_dataset = CodeDiffMessageDataset(train_df, tokenizer)

def custom_collate_fn(
    batch,
    pad_token_id=50256,
    ignore_index=-100,
):
    # Find the longest sequence in the batch
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs and targets
    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])  # Truncate the last token for inputs
        targets = torch.tensor(padded[1:])  # Shift +1 to the right for targets

        # New: Replace all but the first padding tokens in targets by ignore_index
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index


        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Convert list of inputs and targets to tensors and transfer to target device
    device = None
    if torch.cuda.is_available():
      device = torch.device("cuda")
    else:
      device = torch.device("cpu")
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Device:", device)

[50256]
Device: cuda


In [12]:
next(iter(train_loader))

(tensor([[50256,    35, 29267,  ..., 50256, 50256, 50256],
         [50256,    35, 29267,  ...,  1377,  2458, 50256],
         [50256,    35, 29267,  ..., 50256, 50256, 50256],
         [50256,    35, 29267,  ..., 50256, 50256, 50256]], device='cuda:0'),
 tensor([[   35, 29267,    25,  ...,  -100,  -100,  -100],
         [   35, 29267,    25,  ...,  2458, 50256,  -100],
         [   35, 29267,    25,  ...,  -100,  -100,  -100],
         [   35, 29267,    25,  ...,  -100,  -100,  -100]], device='cuda:0'))

In [7]:
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]

batch = (
    inputs_1,
    inputs_2,
    inputs_3
)

print(custom_collate_fn(batch))

(tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]], device='cuda:0'), tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256,  -100,  -100,  -100],
        [    8,     9, 50256,  -100,  -100]], device='cuda:0'))


In [11]:
from torch.utils.data import DataLoader


num_workers = 0
batch_size = 4

torch.manual_seed(123)

train_dataset = CodeDiffMessageDataset(train_df, tokenizer)
val_dataset = CodeDiffMessageDataset(val_df, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=custom_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=custom_collate_fn,
    shuffle= False,
    drop_last=True,
    num_workers=num_workers
)

In [ ]:
#print("Train loader:")
#for inputs, targets in train_loader:
#    print(inputs.shape, targets.shape)

Train loader:
torch.Size([4, 326]) torch.Size([4, 326])
torch.Size([4, 483]) torch.Size([4, 483])
torch.Size([4, 258]) torch.Size([4, 258])
torch.Size([4, 401]) torch.Size([4, 401])
torch.Size([4, 294]) torch.Size([4, 294])
torch.Size([4, 395]) torch.Size([4, 395])
torch.Size([4, 406]) torch.Size([4, 406])
torch.Size([4, 422]) torch.Size([4, 422])
torch.Size([4, 350]) torch.Size([4, 350])
torch.Size([4, 328]) torch.Size([4, 328])
torch.Size([4, 283]) torch.Size([4, 283])
torch.Size([4, 273]) torch.Size([4, 273])
torch.Size([4, 503]) torch.Size([4, 503])
torch.Size([4, 411]) torch.Size([4, 411])
torch.Size([4, 435]) torch.Size([4, 435])
torch.Size([4, 299]) torch.Size([4, 299])
torch.Size([4, 391]) torch.Size([4, 391])
torch.Size([4, 445]) torch.Size([4, 445])
torch.Size([4, 330]) torch.Size([4, 330])
torch.Size([4, 291]) torch.Size([4, 291])
torch.Size([4, 272]) torch.Size([4, 272])
torch.Size([4, 355]) torch.Size([4, 355])
torch.Size([4, 268]) torch.Size([4, 268])
torch.Size([4, 311])

KeyboardInterrupt: 

In [8]:
# 1. Παίρνουμε το πρώτο batch
first_batch = next(iter(train_loader))

# 2. Αποσυμπλέκουμε τα δεδομένα (input_ids και labels)
input_ids, labels = first_batch

print(f"Batch Input Shape: {input_ids.shape}")
print(f"Batch Labels Shape: {labels.shape}")

# 3. Δες το πρώτο δείγμα του batch σε μορφή κειμένου
sample_index = 0
decoded_text = tokenizer.decode(input_ids[sample_index], skip_special_tokens=False)

print("\n--- Πρώτο Δείγμα (Decoded) ---")
print(input_ids[sample_index])

print("\n--- Labels (για να δεις το Masking με -100) ---")
print(labels[sample_index])

Batch Input Shape: torch.Size([4, 237])
Batch Labels Shape: torch.Size([4, 237])

--- Πρώτο Δείγμα (Decoded) ---
tensor([50256,    35, 29267,    25,   198, 25664,    25, 20426,    14,   538,
        39557,   282,    14, 36560,    13,  2188, 25248,   532,  2670,    11,
           21,  1343,  2670,    11,    24, 25248, 25439,   649, 34991,     7,
         1102, 34415,   493,     8,  1635, 36560,  1391,   611,  1673, 13382,
        14512,   657,  1391, 10662,    13,  1102, 22019, 27201,   796,   787,
            7,  3147,  7071,    90,  5512,  1673, 13382,     8,  1343,   329,
         1312, 19039,   657,    26,  1312,  1279,  1673, 13382,    26,  1312,
         4880,  1391,  1343, 10662,    13,  1102, 22019, 27201, 24293, 18038,
         1343,  1782,  1782,  1441, 10662,   198,   198,  9858, 36393,   337,
         1546,  4090,  8264,    25,   198,  1102, 34415,  1630,   329,  2462,
        39557,   282, 20426, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

model.config.pad_token_id = tokenizer.eos_token_id
model.config.use_cache = False
model.config.pad_token_id = model.config.eos_token_id
print(tokenizer.vocab_size)
model.config.pad_token_id





model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

50257


50256

In [10]:
print(model.config)

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "dtype": "float32",
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.57.6",
  "use_cache": false,
  "vocab_size": 50257
}



In [ ]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import math
from transformers import get_linear_schedule_with_warmup
import os


def calculate_accuracy(logits, labels):
    predictions = torch.argmax(logits, dim=-1) #(batch_size, sequence_length, vocab_size)
    correct_predictions = (predictions == labels).float() # (batch_size, sequence_length)
    return correct_predictions.mean().item()

def get_random_validation_diff(val_df):
    # Διαλέγουμε μια τυχαία γραμμή από το validation dataframe
    random_row = val_df.sample(n=1).iloc[0]
    return random_row['diff'], random_row['message']

def generate(model, tokenizer, device,val_df):
    model.eval()
    val_diff, actual_message = get_random_validation_diff(val_df)
    val_diff = clean_text(val_diff)
    actual_message = clean_text(actual_message)
    prompt = f"<|endoftext|>DIFF:\n{val_diff}\n\nCOMMIT MESSAGE:\n"
    inputs = tokenizer(
        prompt, 
        return_tensors="pt",
        max_length=512, 
        truncation=True
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=40,
            do_sample=True,      
            top_p=0.9,
            repetition_penalty=1.2, 
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Αποκωδικοποίηση μόνο της απάντησης
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_message = full_text.split("COMMIT MESSAGE:")[-1].strip()

    print("\n" + "="*50)
    print(f"TEST ON RANDOM VAL DIFF:")
    print(f"DIFF:\n{val_diff}")
    print(f"\nACTUAL MESSAGE: {actual_message}")
    print(f"GENERATED MESSAGE: {generated_message}")
    print("="*50 + "\n")
    model.train()

# --- 2. Training Step ---
def train_one_epoch(model, dataloader, optimizer, device, progress_bar, tokenizer,val_df,start_epoch,epoch,global_step):
    model.train()
    total_loss = 0
    total_acc = 0
    

    for i, (x, y) in enumerate(dataloader):
        print(f"i:{i},global_step:{global_step}")
        if epoch == start_epoch and i < (global_step % len(train_loader)):
                progress_bar.update(1)
                continue

        
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        # Forward pass
        logits = model(x).logits
        loss = nn.CrossEntropyLoss()(
            logits.view(-1, logits.size(-1)),
            y.view(-1)
        )
        acc = calculate_accuracy(logits, y)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        total_acc += acc

        progress_bar.update(1)
        progress_bar.set_postfix({"train_loss": f"{loss.item():.4f}","acc": f"{acc:.4f}","lr": f"{scheduler.get_last_lr()[0]:.2e}"})

        global_step += 1
        if global_step > 0 and global_step % 500 == 0:
            generate(model, tokenizer, device,val_df)
            checkpoint = {
                    'epoch': epoch,
                    'global_step': global_step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict()
                }
            torch.save(checkpoint, "latest_checkpoint.pt")
            print(f"Checkpoint saved at step {global_step}")


    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)
    return avg_loss,avg_acc,global_step

# --- 3. Evaluation Step ---
def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    total_acc=0
    with torch.no_grad():

        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x).logits
            loss = nn.CrossEntropyLoss()(
                logits.view(-1, logits.size(-1)),
                y.view(-1)
            )
            total_loss += loss.item()
            total_acc  += calculate_accuracy(logits, y)


    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)
    return avg_loss,avg_acc


# --- 4. Main Loop ---
def train(model, train_loader, val_loader, optimizer,scheduler, device, tokenizer,epochs,val_df):
    best_val_loss = float("inf")
    patience = 2
    bad_epochs = 0
    checkpoint_path = "latest_checkpoint.pt"
    start_epoch = 0
    global_step = 0

    # ΕΛΕΓΧΟΣ ΓΙΑ RESUME
    if os.path.exists(checkpoint_path):
        print(f"Loading checkpoint: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path)
        
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
        start_epoch = checkpoint['epoch']
        global_step = checkpoint['global_step']
        print(f"Resuming from Epoch {start_epoch}, Global Step {global_step}")

    for epoch in range(start_epoch,epochs):

        progress_bar = tqdm(enumerate(train_loader), 
                            total=len(train_loader), 
                            desc=f"Epoch {epoch+1}/{epochs}",
                            dynamic_ncols=True,
                            mininterval=3.0
                            )

        # Train
        train_loss,train_acc,global_step = train_one_epoch(model, 
                                                          train_loader,    
                                                          optimizer, device, 
                                                          progress_bar, 
                                                          tokenizer,val_df,
                                                          start_epoch,epoch,
                                                          global_step)

        # Evaluate
        val_loss,val_acc = evaluate(model, 
                                    val_loader, 
                                    device)
        try:
            ppl = math.exp(val_loss)
        except OverflowError:
            ppl = float('inf')

        progress_bar.set_postfix({
            "train_loss": f"{train_loss:.4f}",
            "train_accuracy": f"{train_acc:.4f}",
            "val_loss": f"{val_loss:.4f}",
            "val_accuracy": f"{val_acc:.4f}",
            "PPL": f"{ppl:.2f}"
        })
        progress_bar.refresh()
        progress_bar.close() # Close bar to print new line

        # Logging
        print(f"Summary Epoch {epoch+1}: Train Loss: {train_loss:.4f}| Train Accuracy: {train_acc:.4f}\
        | Val Loss: {val_loss:.4f}| Val Accuracy: {val_acc:.4f})| PPL: {ppl:.4f}")

        # Save checkpoints
        torch.save(model.state_dict(), f"{main_path}/gpt2_epoch{epoch+1}.pt")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            bad_epochs = 0
            torch.save(model.state_dict(), f"{main_path}/best_model.pt")
            print(">>> New best model saved!")
        else:
            bad_epochs += 1
            print(f">>> No improvement (bad epochs: {bad_epochs})")

        if bad_epochs >= patience:
            print("EARLY STOPPING TRIGGERED.")
            break

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
lr = 5e-5
epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
num_training_steps = epochs * len(train_loader)
num_warmup_steps = int(0.1 * num_training_steps) 
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=num_warmup_steps, 
    num_training_steps=num_training_steps
)
train(model, train_loader, val_loader, optimizer, scheduler ,device, tokenizer,epochs,val_df)

Loading checkpoint: latest_checkpoint.pt
Resuming from Epoch 0, Global Step 500


Epoch 1/3:   0%|          | 0/230681 [00:00<?, ?it/s]

i:0,global_step:500
i:1,global_step:500
i:2,global_step:500
i:3,global_step:500
i:4,global_step:500
i:5,global_step:500
i:6,global_step:500
i:7,global_step:500
i:8,global_step:500
i:9,global_step:500
i:10,global_step:500
i:11,global_step:500
i:12,global_step:500
i:13,global_step:500
i:14,global_step:500
i:15,global_step:500
i:16,global_step:500
i:17,global_step:500
i:18,global_step:500
i:19,global_step:500
i:20,global_step:500
i:21,global_step:500
i:22,global_step:500
i:23,global_step:500
i:24,global_step:500
i:25,global_step:500
i:26,global_step:500
i:27,global_step:500
i:28,global_step:500
i:29,global_step:500
i:30,global_step:500
i:31,global_step:500
i:32,global_step:500
i:33,global_step:500
i:34,global_step:500
i:35,global_step:500
i:36,global_step:500
i:37,global_step:500
i:38,global_step:500
i:39,global_step:500
i:40,global_step:500
i:41,global_step:500
i:42,global_step:500
i:43,global_step:500
i:44,global_step:500
i:45,global_step:500
i:46,global_step:500
i:47,global_step:500
i:

KeyboardInterrupt: 

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Ορισμός συσκευής και μοντέλου βάσης
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "gpt2"

# 2. Φόρτωση του tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. Αρχικοποίηση του μοντέλου (Base GPT-2)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 4. Φόρτωση των δικών σου βαρών από το .pt αρχείο
checkpoint_path = 'data/best_model.pt'
state_dict = torch.load(checkpoint_path, map_location=device)

# Φόρτωση των βαρών στο μοντέλο
model.load_state_dict(state_dict)

model.to(device)
model.eval()

print("Το εκπαιδευμένο μοντέλο φορτώθηκε επιτυχώς!")

Το εκπαιδευμένο μοντέλο φορτώθηκε επιτυχώς!


In [25]:
def generate_commit_message(diff_text, model, tokenizer, device):
    model.eval()
    
    # 1. Καθαρισμός και σωστό Format (ίδιο με το training!)
    clean_diff = (diff_text) 
    prompt = f"<|endoftext|>DIFF:\n{clean_diff}\n\nCOMMIT MESSAGE:\n"
    
    # 2. Tokenization
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 3. Παραγωγή (Generation)
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=50,      
            do_sample=True,        
            top_p=0.92,             
            temperature=0.7,        
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2  
        )
    
    # 4. Decoding
    full_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
    # Επιστρέφουμε μόνο το κομμάτι μετά το "COMMIT MESSAGE:"
    message = full_text.split("COMMIT MESSAGE:")[-1].strip()
    return message

test_diff = """--- a/app.py
+++ b/app.py
- def calculate(a, b): return a+b
+ def add_numbers(a, b): return a + b"""
print(generate_commit_message(test_diff,model,tokenizer,device))
    

Increment numbers to use numbers instead of numbers for number of units.


In [11]:
import torch
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Setup the App
app = FastAPI(title="Git Diff to Commit Message API")

# 2. Define Request Schema
# This ensures users send valid JSON with a "diff" field
class DiffRequest(BaseModel):
    diff: str
    max_tokens: int = 50  # Default value, but user can change it

# 3. Global Variables for Model (Loaded on Startup)
model = None
tokenizer = None
device = None

@app.on_event("startup")
def load_model():
    global model, tokenizer, device

    # Configuration
    MODEL_PATH = "best_model.pt" # Or your specific checkpoint path
    BASE_MODEL_NAME = "gpt2"     # Needed for the tokenizer configuration

    print("Loading model and tokenizer...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Tokenizer (Use the base gpt2 tokenizer)
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token

    # Load Model Structure & Weights
    model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME)
    model.resize_token_embeddings(len(tokenizer)) # Important if you resized during training

    # Load your fine-tuned weights
    # map_location ensures it loads even if you move from GPU -> CPU
    state_dict = torch.load(MODEL_PATH, map_location=device)
    model.load_state_dict(state_dict)

    model.to(device)
    model.eval()
    print(f"Model loaded successfully on {device}!")

# 4. Generation Endpoint
@app.post("/generate")
async def generate_commit_message(request: DiffRequest):
    if not model:
        raise HTTPException(status_code=500, detail="Model not loaded")

    try:
        # Format the input exactly how we trained it
        generate_commit_message(DiffRequest,model,tokenizer,device)

        return {
            "generated_message": answer,
            "full_text": full_text
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# 5. Health Check
@app.get("/health")
def health_check():
    return {"status": "ok", "device": str(device)}

/tmp/ipython-input-463976673.py:20: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [27]:
import requests

url = "http://127.0.0.1:8000/generate"
data = {"diff": "--- a/file.py\n+++ b/file.py\n- print('error')\n+ print('fixed')"}

response = requests.post(url, json=data)
print(response.json()['generated_message'])

fix(file) fix error handling
